# 1. Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join, abspath, dirname
from tqdm import tqdm
import random

import numpy as np
import pandas as pd
from dask import delayed, compute
from dask.diagnostics import ProgressBar
import sklearn
import cv2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pytorch_lightning as pl

/opt/conda/envs/patchcore/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/patchcore/lib/python3.6/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /opt/conda/envs/patchcore/lib/python3.6/site-packages/torchvision/image.so: undefined symbol: _ZNK3c106IValue23reportToTensorTypeErrorEv
  warn(f"Failed to load image Python extension: {e}")


In [3]:
class PATH:
    root   = abspath(dirname('.'))
    input  = join(root, 'open')
    train  = join(input, 'train')
    test   = join(input, 'test')
    embedding = join(root, 'embedding')
    
TEST_SIZE   = 0.3
LABEL1      = 'class'
LABEL2      = 'state'
LABEL3      = 'label'
SIZE        = 64
CROP_SIZE   = 60
INPUT_SHAPE = (SIZE, SIZE, 3)
BATCH_SIZE  = 32
GPUS        = 1

RANDOM_STATE = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    sklearn.random.seed(RANDOM_STATE)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    pl.seed_everything(RANDOM_STATE, workers=True)
seed_everything(RANDOM_STATE)

Global seed set to 42


In [4]:
from util import *

# 2. Load dataset

In [5]:
train_full_data_meta = pd.read_csv(join(PATH.input, 'train_df.csv'), index_col=0)
test_data_meta       = pd.read_csv(join(PATH.input, 'test_df.csv'), index_col=0)

# 3. Training

## 1) Classification - label

In [6]:
# %%time
# from sklearn.model_selection import train_test_split

# train_data_meta1, val_data_meta1 = train_test_split(train_full_data_meta, stratify=train_full_data_meta[LABEL1])
# X_train1, X_val1 = X_train_full[train_data_meta1.index], X_train_full[val_data_meta1.index]

# y_enc1     = LabelEncoder()
# y_train1   = y_enc1.fit_transform(train_data_meta1[LABEL1])
# y_val1     = y_enc1.transform(val_data_meta1[LABEL1])
# N_CLASSES1 = len(y_enc1.classes_)

# dl_train1 = generate_dataloader(X_train1, y_train1, batch_size=BATCH_SIZE, shuffle=True)
# dl_val1   = generate_dataloader(X_val1, y_val1, batch_size=BATCH_SIZE)
# dl_test1  = generate_dataloader(X_test, batch_size=1)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_data_meta1, val_data_meta1 = train_test_split(train_full_data_meta, stratify=train_full_data_meta[LABEL1])
test_data_meta1 = test_data_meta

y_enc1 = LabelEncoder()
train_data_meta1[LABEL1] = y_enc1.fit_transform(train_data_meta1[LABEL1])
val_data_meta1[LABEL1]   = y_enc1.transform(val_data_meta1[LABEL1])
meta_datas1 = {'train': train_data_meta1, 'val': val_data_meta1, 'test': test_data_meta1}

{k: len(v) for k, v in meta_datas1.items()}

/opt/conda/envs/patchcore/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/envs/patchcore/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


{'train': 3207, 'val': 1070, 'test': 2154}

In [7]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

hparams1 = {
    'n_classes': train_full_data_meta['class'].nunique(),
    'load_size': SIZE,
    'crop_size': CROP_SIZE,
    'label': LABEL1,
    'input_dir': PATH.input,
    'batch_size': BATCH_SIZE
}
model1  = BaseModel(meta_datas1, hparams1)
trainer = pl.Trainer(callbacks=[EarlyStopping(monitor='val_loss', patience=5)], gpus=1, max_epochs=10)
trainer.fit(model1)

/opt/conda/envs/patchcore/lib/python3.6/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | model   | EfficientNet | 4.0 M 
1 | F1Score | F1Score      | 0     
-----------------------------------------
19.2 K    Trainable params
4.0 M     Non-trainable params
4.0 M     Total params
16.107    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]Validation dataset: 1070
Validation sanity check:  50%|█████     | 1/2 [00:00<00:00,  1.54it/s]
  > val_loss: tensor(6.4053, device='cuda:0') 	 val_f1_score: tensor(0., device='cuda:0')
                                                                      

Global seed set to 42


Training dataset: 3207
Epoch 0:   9%|▊         | 101/1171 [00:20<03:36,  4.95it/s, loss=0.384, v_num=58]
Validating: 0it [00:00, ?it/s]
Validating:  99%|█████████▉| 1063/1070 [00:10<00:00, 132.32it/s]
  > val_loss: tensor(0.2916, device='cuda:0') 	 val_f1_score: tensor(0.9150, device='cuda:0')
Epoch 0: 100%|██████████| 1171/1171 [00:31<00:00, 37.77it/s, loss=0.384, v_num=58]
                                                                
  > training_loss: tensor(1.4375, device='cuda:0') 	 training_f1_score: tensor(0.6239, device='cuda:0')


Epoch 1:   9%|▊         | 102/1171 [00:14<02:35,  6.88it/s, loss=0.107, v_num=58] 
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████▉| 1069/1070 [00:09<00:00, 135.92it/s]
  > val_loss: tensor(0.1153, device='cuda:0') 	 val_f1_score: tensor(0.9720, device='cuda:0')
Epoch 1: 100%|██████████| 1171/1171 [00:24<00:00, 48.12it/s, loss=0.107, v_num=58]
                                                                
  > training_loss: tensor(0.2007, device='cuda:0') 	 training_f1_score: tensor(0.9083, device='cuda:0')
Epoch 2:   9%|▊         | 102/1171 [00:14<02:36,  6.84it/s, loss=0.151, v_num=58] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  19%|█▉        | 221/1171 [00:16<01:10, 13.44it/s, loss=0.151, v_num=58]


...

Validating:  99%|█████████▉| 1061/1070 [00:09<00:00, 136.11it/s]
  > val_loss: tensor(0.0521, device='cuda:0') 	 val_f1_score: tensor(0.9832, device='cuda:0')
Epoch 5: 100%|██████████| 1171/1171 [00:24<00:00, 47.52it/s, loss=0.0882, v_num=58]
                                                                
  > training_loss: tensor(0.0543, device='cuda:0') 	 training_f1_score: tensor(0.9704, device='cuda:0')
Epoch 6:   9%|▊         | 102/1171 [00:15<02:40,  6.66it/s, loss=0.0656, v_num=58] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  54%|█████▎    | 629/1171 [00:20<00:17, 30.84it/s, loss=0.0656, v_num=58]


Validating: 100%|█████████▉| 1069/1070 [00:09<00:00, 141.15it/s]
  > val_loss: tensor(0.0271, device='cuda:0') 	 val_f1_score: tensor(0.9925, device='cuda:0')
Epoch 6: 100%|██████████| 1171/1171 [00:25<00:00, 46.64it/s, loss=0.0656, v_num=58]
                                                                
  > training_loss: tensor(0.0582, device='cuda:0') 	 training_f1_score: tensor(0.9685, device='cuda:0')
Epoch 7:   9%|▊         | 102/1171 [00:14<02:34,  6.90it/s, loss=0.0793, v_num=58] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  62%|██████▏   | 731/1171 [00:20<00:12, 35.30it/s, loss=0.0793, v_num=58]


Validating:  99%|█████████▉| 1057/1070 [00:09<00:00, 134.23it/s]
  > val_loss: tensor(0.0290, device='cuda:0') 	 val_f1_score: tensor(0.9888, device='cuda:0')
Epoch 7: 100%|██████████| 1171/1171 [00:24<00:00, 47.71it/s, loss=0.0793, v_num=58]
                                                                
  > training_loss: tensor(0.0568, device='cuda:0') 	 training_f1_score: tensor(0.9675, device='cuda:0')
Epoch 8:   9%|▊         | 102/1171 [00:15<02:39,  6.72it/s, loss=0.185, v_num=58]  
Validating: 0it [00:00, ?it/s]
Epoch 8:  73%|███████▎  | 850/1171 [00:22<00:08, 38.40it/s, loss=0.185, v_num=58]


Validating: 100%|██████████| 1070/1070 [00:09<00:00, 132.48it/s]
  > val_loss: tensor(0.0200, device='cuda:0') 	 val_f1_score: tensor(0.9944, device='cuda:0')
Epoch 8: 100%|██████████| 1171/1171 [00:24<00:00, 47.13it/s, loss=0.185, v_num=58]
                                                                
  > training_loss: tensor(0.0780, device='cuda:0') 	 training_f1_score: tensor(0.9700, device='cuda:0')
Epoch 9:   9%|▊         | 102/1171 [00:14<02:36,  6.84it/s, loss=0.105, v_num=58] 
Validating: 0it [00:00, ?it/s]
Epoch 9:  81%|████████▏ | 952/1171 [00:22<00:05, 41.45it/s, loss=0.105, v_num=58]


Validating:  99%|█████████▉| 1061/1070 [00:09<00:00, 137.59it/s]
  > val_loss: tensor(0.0279, device='cuda:0') 	 val_f1_score: tensor(0.9944, device='cuda:0')
Epoch 9: 100%|██████████| 1171/1171 [00:24<00:00, 47.14it/s, loss=0.105, v_num=58]
                                                                
  > training_loss: tensor(0.0492, device='cuda:0') 	 training_f1_score: tensor(0.9779, device='cuda:0')
Epoch 9: 100%|██████████| 1171/1171 [00:24<00:00, 46.99it/s, loss=0.105, v_num=58]


In [ ]:
pred1 = trainer.predict(model1, dl_test1)
test_data_meta['class'] = y_enc1.inverse_transform(pred1)
test_data_meta

## 2) Classification - state (good or not)

In [ ]:
def get_dataset2(class_, train_full_data_meta, test_data_meta, X_train_full):
    idxs      = train_full_data_meta.query(f"`class` == '{class_}'").index
    idxs_test = test_data_meta.query(f"`class` == '{class_}'").index
    
    train_full_data_meta2 = train_full_data_meta.loc[idxs]
    train_data_meta2, val_data_meta2 = train_test_split(train_full_data_meta2, stratify=train_full_data_meta2[LABEL2])
    
    X_train2, X_val2 = X_train_full[train_data_meta2.index], X_train_full[val_data_meta2.index]
    X_test2          = X_test[idxs_test]
    
    y_train2   = train_data_meta2[LABEL2] != 'good'
    y_val2     = val_data_meta2[LABEL2]   != 'good'
    
    dl_train2 = generate_dataloader(X_train2, y_train2, batch_size=BATCH_SIZE, shuffle=True)
    dl_val2   = generate_dataloader(X_val2, y_val2, batch_size=BATCH_SIZE)
    dl_test2  = generate_dataloader(X_test2, batch_size=1)
    return dl_train2, dl_val2, dl_test2, idxs_test

In [ ]:
N_CLASSES2 = 2
for class_ in y_enc1.classes_:
    _, _, dl_test2, idxs_test = get_dataset2(class_, train_full_data_meta, test_data_meta, X_train_full)
    args = {
        'batch_size': BATCH_SIZE,
        'load_size': SIZE,
        'input_size': 224,
        'coreset_sampling_ratio': 0.0001,
        'n_neighbors': 9,
        'gpus': 1,
        'max_epochs': 1,
        'category': class_,
        'input_dir': PATH.input,
        'embedding_dir': PATH.embedding,
    }
    trainer = pl.Trainer(gpus=args['gpus'], max_epochs=args['max_epochs'])
    model   = STPM(hparams=args, meta_datas=split_meta_data(args['category'], args['input_dir']))
    trainer.fit(model)
    trainer.test(model)  # validation
    test_data_meta.at[idxs_test, LABEL2] = trainer.predict(model, dl_test2)
#     break

In [ ]:
test_data_meta.query(f"`class` == '{class_}'").head()

## 3) Classification - label

In [ ]:
def get_dataset3(class_, train_full_data_meta, test_data_meta, X_train_full, X_test):
    idxs      = train_full_data_meta.query(f"`class` == '{class_}' and state != 'good'").index
    idxs_test = test_data_meta.query(f"`class` == '{class_}' and state != 0").index
    
    train_full_data_meta = train_full_data_meta.loc[idxs]
    train_data_meta, val_data_meta = train_test_split(train_full_data_meta, stratify=train_full_data_meta[LABEL3])
    
    X_train, X_val = X_train_full[train_data_meta.index], X_train_full[val_data_meta.index]
    X_test         = X_test[idxs_test]
    
    y_enc   = LabelEncoder()
    y_train = y_enc.fit_transform(train_data_meta[LABEL3])
    y_val   = y_enc.transform(val_data_meta[LABEL3])
    
    dl_train = generate_dataloader(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
    dl_val   = generate_dataloader(X_val, y_val, batch_size=BATCH_SIZE)
    dl_test  = generate_dataloader(X_test, batch_size=1)
    return dl_train, dl_val, dl_test, idxs_test, y_enc

In [ ]:
for class_ in y_enc1.classes_:
    dl_train3, dl_val3, dl_test3, idxs_test3, y_enc3 = get_dataset3(class_, train_full_data_meta, test_data_meta, X_train_full, X_test)
    model3  = BaseModel(len(y_enc3.classes_))
    trainer = pl.Trainer(callbacks=[EarlyStopping(monitor='val_loss', patience=5)], gpus=1, max_epochs=5)
    trainer.fit(model3, dl_train3, dl_val3)
    pred3 = trainer.predict(model3, dl_test3)
    
    test_data_meta[LABEL3]                = test_data_meta.apply(lambda row: f"{row['class']}-good", axis='columns')
    test_data_meta.at[idxs_test3, LABEL3] = y_enc3.inverse_transform(pred3)
#     break

In [ ]:
test_data_meta.query(f"`class` == '{class_}'")